In [1]:
import datetime
import json
import os
import pandas as pd
import logging


In [2]:
from utils.data_transformation import prepare_dialogs, if_in_date_range, prepare_dialogs_sorted_by_lang

In [3]:
!pip install -r requirements.txt

ERROR: Could not detect requirement name for 'git+https://github.com/Desklop/Uk_Stemmer', please specify one with #egg=your_package_name


In [4]:
# TODO: add this packages to the requirements.txt file

# !pip install -U nlpcube
# !pip install stop-words
# !pip install git+https://github.com/Desklop/Uk_Stemmer

In [5]:
# TODO: rename this method, calculate is to generic word, name should give you information about method
def get_sorted_word_frequency(dialog_data, start_date, end_date, user_id_get_msg, dialog_id, path_to_processed_files):
    """
    :param dialog_data: a pandas dataframe of dialogs
    :param start_date: datetime type, from what time start to analyse msgs
    :param end_date: datetime type, to what time to analyse msgs
    :param user_id_get_msg: a user chat id, who msgs to analyse or "all" - for all participants of the dialog
    :param dialog_id: str
    :param path_to_processed_files: path to processed_dialog_files dir
    :return: a sorted df - sorted dict of user_id_get_msg most used words
    """
    DF = {}
    file_text = ""

    for row in dialog_data.index:
        if user_id_get_msg != "all":
            if dialog_data["from_id"][row] != user_id_get_msg:
                continue

        if if_in_date_range(dialog_data["date"][row][:-6], start_date, end_date) == 'Dialog after end_date':
            continue

        elif if_in_date_range(dialog_data["date"][row][:-6], start_date, end_date):
            # TODO: copy this part of function(@rey_broncas)
            if not pd.isnull(dialog_data["message"][row]):
                file_text += dialog_data["message"][row] + " "
                for w in dialog_data["message"][row].split():
                    try:
                        DF[w] += 1
                    except:
                        DF[w] = 1
        else:
            break

    with open(path_to_processed_files + "{}/{}.txt".format(dialog_id, dialog_id), "w", encoding="utf-8") as f:
        f.write(file_text)

    DF_sorted = {k: v for k, v in sorted(DF.items(), key=lambda item: item[1], reverse=True)}
    
    return DF_sorted

In [6]:
# 1) change a dialog_id, which you want to investigate
dialog_id = "620593584"

# 2) change user_id_get_msg
# int type - user chat id, who massages you want to analyse,
# you can find it in <dialog_id>.csv
# str type "all" if you want to analyse msgs of all users of the dialog
user_id_get_msg = 511986933
dialog_path = "data/dialogs/"
prep_path = 'data/prepared_dialogs/'
# dialog_lang = "ua"
path_to_logs = os.path.join("logs", "project_logs.log")

# TODO: path_to_static, static what? you should rename this folder with respect to its purpose!
path_to_processed_files = "data/processed_dialog_files/"

# TODO: rename start_date and end_date -> start_date and end_date
# 3) change to date range in what you want to analyse messages of user_id_get_msg - from start_date to end_date;
# format "%Y-%m-%d %H:%M:%S"
start_date = datetime.datetime(2017, 7, 9, 0, 0, 0)
end_date = datetime.datetime(2020, 8, 10, 0, 0, 0)

In [7]:
# TODO: add log file path to global var
logging.basicConfig(filename=path_to_logs, level=0)
logging.info("starting logs for tf_idf_dialogs")

# TODO: move instructions in the README file

# 0) in console input: import nltk; nltk.download()
#
# Install (or update) NLP-Cube with:
# pip3 install -U nlpcube
#
# use telegram-data-collection/0_download_dialogs_list.py and 1_download_dialogs_data.py
# to get some files to analyse
# files SHOULD be in data/dialogs and data/dialogs_meta

In [8]:
if not os.path.exists(path_to_processed_files):
    os.mkdir(path_to_processed_files)
    
if not os.path.exists(path_to_processed_files + dialog_id):
    os.mkdir(path_to_processed_files + dialog_id)

In [9]:
# TODO: create a method to detect dialog language

# 4) change "ua" to a language of your dialog ("ua", "ru" or "en")
# if you write "ua" or "ru" - dialog will be cleaned from stop_words_ua + stop_words_ru + stop_words_en
# so do not bother about these languages if you can not write an exactly language of the dialog

# TODO: move lang to the variable
prepare_dialogs_sorted_by_lang([dialog_id], dialog_path, prep_path, start_date, end_date)
# prepare_dialogs(dialog_id, dialog_path, prep_path, start_date,
#                 end_date, dialog_lang, "words_frequency")

dialog_ids_sorted_by_lang
{'en': [], 'ru': [], 'ua': ['620593584']}
Loading latest local model: uk-1.1
	Loading embeddings ... 
	Loading tokenization model ...
	Loading lemmatization model ...
	Loading tagger model ...
	Loading parser model ...
Model loading complete.


=======Language ua -- 1 from 1=======
INDEX 3
INDEX 4
INDEX 5
INDEX 6
INDEX 7
INDEX 8
INDEX 9
INDEX 10
INDEX 11
INDEX 12
INDEX 13
INDEX 14
INDEX 15
INDEX 16
INDEX 17
INDEX 18
INDEX 19
INDEX 20
INDEX 21
INDEX 22
INDEX 23
INDEX 24
INDEX 25
INDEX 26
INDEX 27
INDEX 28
INDEX 29
INDEX 31
INDEX 32
INDEX 33
INDEX 34
INDEX 35
INDEX 36
INDEX 37
INDEX 38
INDEX 39
INDEX 40
INDEX 41
INDEX 42
INDEX 43
INDEX 44
INDEX 45
INDEX 46
INDEX 47
INDEX 48
INDEX 49
INDEX 50
INDEX 51
INDEX 52
INDEX 53
INDEX 54
INDEX 55
INDEX 56
INDEX 57
INDEX 58
INDEX 59
INDEX 60
INDEX 61
INDEX 62
INDEX 63
INDEX 64
INDEX 65
INDEX 66
INDEX 67
INDEX 68
INDEX 69
INDEX 70
INDEX 71
INDEX 72
INDEX 73
INDEX 74
INDEX 75
INDEX 76
INDEX 77
INDEX 78
INDEX 79
INDEX 80
INDEX

In [10]:
dialog_data = pd.read_csv(prep_path + dialog_id + ".csv")

DF_sorted = get_sorted_word_frequency(dialog_data, start_date, end_date, user_id_get_msg, dialog_id, path_to_processed_files)

In [11]:
with open(path_to_processed_files + "{}/words_frequency_{}.json".format(dialog_id, dialog_id), "w", encoding="utf-8") as f:
    json.dump(DF_sorted, f, indent=4, ensure_ascii=False)

# 5) after work of this module look at results in static/<dialog_id> dir,
# use command under to make a wordcloud, to understand its parameters use - wordcloud_cli --help
# Example: wordcloud_cli --text static/<dialog_id>/1_<dialog_id>.txt --mask static/logo_telegram.jpg --imagefile static/my_words_nazar2.png
print(DF_sorted)

{'привіт': 32, 'христя': 29, 'могти': 25, 'річ': 15, 'проект': 13, 'курс': 11, 'мати': 10, 'хотіти': 10, 'якийсь': 9, 'дякувати': 9, 'дуже': 9, 'скинути': 8, 'квартира': 7, 'запитати': 7, 'ласок': 7, 'який': 7, 'навчання': 6, 'вивчати': 6, 'лекція': 6, 'запитувати': 5, 'що': 5, 'зараз': 5, 'круто': 5, 'порада': 5, 'добре': 5, 'історія': 5, 'рік': 5, 'іспит': 5, 'підсказати': 5, 'знімати': 4, 'давати': 4, 'щось': 4, 'використовувати': 4, 'інформація': 4, 'сподіватися': 4, 'написати': 4, 'рецензія': 4, 'говорити': 3, 'курся': 3, 'програмування': 3, 'знайти': 3, 'якщо': 3, 'пройти': 3, 'завтра': 3, 'акаунт': 3, 'думати': 3, 'запис': 3, 'захист': 3, 'успіх': 3, 'поглянути': 3, 'новина': 3, 'текст': 3, 'код': 3, 'бати': 3, 'готуватися': 3, 'завдання': 3, 'точно': 3, 'зробити': 3, 'підказати': 3, 'тест': 3, 'помилка': 3, 'робота': 3, 'програма': 3, 'семінар': 3, 'книга': 3, 'серпень': 2, 'починати': 2, 'десь': 2, 'інший': 2, 'взагалі': 2, 'ук': 2, 'під': 2, 'вигідно': 2, 'термін': 2, 'більши